In [157]:
import pandas as pd
df = pd.read_csv('titanic.csv')

In [158]:
df.rename(columns= {
    'Siblings/Spouses Aboard':'ssa',
    'Parents/Children Aboard': 'pca'
}, inplace=True)

def func(count):
    if count>0:
        return "Yes"
    else:
        return "No"
    
def age(count):
    if count < 20:
        return "Child"
    else:
        return "Adult"

def func1(count):
    if count>1:
        return 2
    else:
        return 1
    
df['Age']=df['Age'].apply(age)
df['pca']=df['pca'].apply(func)
df['ssa']=df['ssa'].apply(func)
df['Pclass']=df['Pclass'].apply(func1)

In [159]:
df.drop(columns=['Name','Fare'], inplace=True)
df

,Survived,Pclass,Sex,Age,ssa,pca
0,0,2,male,Adult,Yes,No
1,1,1,female,Adult,Yes,No
2,1,2,female,Adult,No,No
3,1,1,female,Adult,Yes,No
4,0,2,male,Adult,No,No
...,...,...,...,...,...,...
882,0,2,male,Adult,No,No
883,1,1,female,Child,No,No
884,0,2,female,Child,Yes,Yes
885,1,1,male,Adult,No,No


In [160]:
subset = df[['Pclass', 'Sex', 'Age', 'ssa', 'pca','Survived']]
X = [tuple(x) for x in subset.values]
X_train = X[5:10]
X_test = X[10:]

In [162]:
def g_0(n):
    return ("?",)*n

def s_0(n):
    return ('0',)*n

In [163]:
def more_general(h1, h2):
    more_general_parts = []
    for x, y in zip(h1, h2):
        mg = x == '?' or (x != '0' and (x == y or y == '0'))
        more_general_parts.append(mg)
    return all(more_general_parts)

def consistent(hypothesis,example):
    return more_general(hypothesis, example)

def get_domains(examples):
    d = [set() for i in examples[0]] 
    for x in examples:
        for i, xi in enumerate(x):
            d[i].add(xi)
    return [list(sorted(x)) for x in d]

In [164]:
def min_generalizations(h, x):
    h_new = list(h)
    for i in range(len(h)):
        if not consistent(h[i:i+1],x[i:i+1]):
            if h[i] != '0':
                h_new[i] = '?'
            else:
                h_new[i] = x[i]
    return [tuple(h_new)]

def generalize_S(x, G, S):
    S_prev = list(S)
    for s in S_prev:
        if s not in S:
            continue
        if not consistent(s,x):
            S.remove(s)
            Splus = min_generalizations(s, x)
            S.update([h for h in Splus if any([more_general(g,h) 
                                               for g in G])])
            S.difference_update([h for h in S if 
                                 any([more_general(h, h1) 
                                      for h1 in S if h != h1])])
    return S

In [165]:
def min_specializations(h, domains, x):
    results = []
    for i in range(len(h)):
        if h[i] == '?':
            for val in domains[i]:
                if x[i] != val:
                    h_new = h[:i] + (val,) + h[i+1:]
                    results.append(h_new)
        elif h[i] != '0':
            h_new = h[:i] + ('0',) + h[i+1:]
            results.append(h_new)
    return results

def specialize_G(x, domains, G, S):
    G_prev = list(G)
    for g in G_prev:
        if g not in G:
            continue
        if consistent(g,x):
            G.remove(g)
            Gminus = min_specializations(g, domains, x)
            G.update([h for h in Gminus if any([more_general(h, s)
                                                for s in S])])
            G.difference_update([h for h in G if 
                                 any([more_general(g1, h) 
                                      for g1 in G if h != g1])])
    return G

In [166]:
def candidate_elimination(examples):
    domains = get_domains(examples)[:-1]
    
    G = set([g_0(len(domains))])
    S = set([s_0(len(domains))])
    i=0
    print("\n G[{0}]:".format(i),G)
    print("\n S[{0}]:".format(i),S)
    for xcx in examples:
        i=i+1
        x, cx = xcx[:-1], xcx[-1]  # Splitting data into attributes and decisions
        if cx==1: # x is positive example
            G = {g for g in G if consistent(g,x)}
            S = generalize_S(x, G, S)
        else: # x is negative example
            S = {s for s in S if not consistent(s,x)}
            G = specialize_G(x, domains, G, S)
        print("\n G[{0}]:".format(i),G)
        print("\n S[{0}]:".format(i),S)
    return G,S
G, S = candidate_elimination(X_train)


 G[0]: {('?', '?', '?', '?', '?')}

 S[0]: {('0', '0', '0', '0', '0')}

 G[1]: {(1, '?', '?', '?', '?'), ('?', '?', '?', 'Yes', '?'), ('?', '?', 'Child', '?', '?'), ('?', 'female', '?', '?', '?'), ('?', '?', '?', '?', 'Yes')}

 S[1]: {('0', '0', '0', '0', '0')}

 G[2]: {('?', '?', '?', 'Yes', '?'), ('?', '?', 'Child', '?', '?'), ('?', 'female', '?', '?', '?'), ('?', '?', '?', '?', 'Yes')}

 S[2]: {('0', '0', '0', '0', '0')}

 G[3]: {('?', '?', '?', 'Yes', 'No'), ('?', '?', 'Child', '?', 'No'), ('?', '?', 'Adult', '?', 'Yes'), (1, '?', 'Child', '?', '?'), ('?', '?', 'Child', 'No', '?'), (1, '?', '?', '?', 'Yes'), ('?', '?', '?', 'No', 'Yes'), (1, '?', '?', 'Yes', '?'), ('?', 'female', '?', '?', '?'), ('?', '?', 'Adult', 'Yes', '?')}

 S[3]: {('0', '0', '0', '0', '0')}

 G[4]: {('?', '?', 'Adult', '?', 'Yes'), ('?', '?', '?', 'No', 'Yes'), ('?', 'female', '?', '?', '?')}

 S[4]: {(2, 'female', 'Adult', 'No', 'Yes')}

 G[5]: {('?', 'female', '?', '?', '?')}

 S[5]: {(2, 'female', '?', '?

In [167]:
def check(instance_attr_val, constraint):
    if (constraint == '-'):
        return False
    elif ((constraint != '?') and (constraint != instance_attr_val)):
        return False
    return True

def test(X_test,hypothesis):
    n = len(X_test)
    m = len(X_test[1]) - 1
    test_passed_cnt = 0
    for i in range(n):
        for j in range(m):
            instance_attr_val = X_test[i][j]
            constraint = hypothesis[j]
            if(check(instance_attr_val, constraint) == False):
                isSatisfying = 0
                break
            else:
                isSatisfying = 1
        if X_test[i][-1] == isSatisfying:
            test_passed_cnt += 1
    accuracy = (test_passed_cnt/n)*100
    return accuracy

In [168]:
print("Accuracy is {}".format(test(X_test,list(S))))

Accuracy is 61.57354618015963
